In [ ]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 112.3 MB/s eta 0:00:00


In [ ]:
import os   #let's us to interact with computer OS. eg, create/delete folders,read files,check file paths etc...
import io   #stands for i/p o/p .Let's us work with data as if it's a file(even if it's in a memory)
import re   #Regular expression. helps to find or replace pattern in text
import json  #used to work with JSON data(common format to store data like a dict)
import pandas as pd  #pandas-->data handling module, popular library for data analysis,works with table(rows and column) like excel
import numpy as np  #numpy-->data handling module,library for numerical calculation,great for working with arrays,matrics and math operation
import time  #let's us work with time(pause pgm,measure how long something takes)
import streamlit as st  #stramlit-->app & dashboard,used to make web apps for DS & ML
from typing import List,Dict,Tuple,Any  #type hinting(not code execution,just for clarity),helps to explain datatypes in function

e.g for type hinting

def add_numbers(numbers: List[int]) -> int:
   return sum(numbers)

list[int] means the input is a list of integer
-> int means the function returns as integer

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
from pypdf import PdfReader
from docx import Document as DocxDocument

NLP & AI models
1.from sentence_transformers import SentenceTransformer
   library to create sentence embbedding(turns sentence into num vector);useful for sementaic search,recommandation,clustering
   Example:
   "I love cats" → [0.12, -0.55, 0.88, ...] (vector).

2.from transformers import pipeline
   Hugging Face Transformers library.
   pipeline is a shortcut to use pre-trained AI models easily.
   Example tasks: text summarization, translation, sentiment analysis, question answering.
   Example:
   from transformers import pipeline
   summarizer = pipeline("summarization")
   print(summarizer("I love tom"))

Searching Similar Data
1.import faiss
  FAISS (by Facebook/Meta) is for fast similarity search.
  Helps you search quickly through millions of vectors (like sentence embeddings).
  Example: find the most similar sentence/document to a query.

Reading Files
1.from pypdf import PdfReader
  Library to read text from PDF files.
  Example: load a PDF, get number of pages, extract text.

from docx import Document as DocxDocument
Library to read and write Word documents (.docx).
Example: open a .docx file and read paragraphs, or create a new Word file.


In short:

SentenceTransformer → turns sentences into numbers (vectors).
pipeline → quick way to use AI models (summarize, translate, classify).
faiss → finds similar vectors really fast (used for search).
PdfReader → extracts text from PDFs.
DocxDocument → reads/writes Word docs.

In [ ]:
pip install faiss-cpu

In [ ]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 7.4 MB/s eta 0:00:00


In [ ]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.3 MB/s eta 0:00:00


In [ ]:
#clean & chunk text
def clean_text(text:str)->str:
  text=re.sub(r"\s+"," ",text)
  return text.strip()

In [ ]:
def chunk_text(text:str, chunk_size:int=800, overlap:int=120) -> List[str]:
  """
  Character based chunking(simple & Robust)
  Chunk size ~800 chars works well for small model like FLAN-T5
  """
  text = clean_text(text)
  chunks = []
  start = 0
  n = len(text)
  while start < n:
    end = min(start + chunk_size, n)
    chunk = text[start:end]
    chunks.append(chunk)
    start=end-overlap
    if start < 0:
      start = 0
  return chunks

In [ ]:
#file loader(PDF,DOCX,CSV,TXT)

In [ ]:
def load_txt(file_bytes: bytes)->str:
  return file_bytes.decode("utf-8",errors="ignore")

In [ ]:
def load_pdf(file_bytes: bytes)->str:
  with io.BytesIO(file_bytes) as fb:
    reader = PdfReader(fb)
    texts=[]
    for page in reader.pages:
      try:
        t=page.extract_text() or ""
      except Exception :
        t=""
      if t:
        texts.append(t)
  return "\n".join(texts)

In [ ]:
def load_docx(file_bytes: bytes)->str:
  with io.BytesIO(file_bytes) as fb:
    doc=DocxDocument(fb)
  return "/n".join(p.text for p in doc.paragraphs)

In [ ]:
def load_csv(file_bytes: bytes)->str:
  with io.BytesIO(file_bytes) as fb:
    df=pd.read_csv(fb)
    #converting to readable FAQ-like table text
  return df.to_csv(index=False)

In [ ]:
def read_any(file)->Tuple[str,str]:
  name=file.name.lower()
  content=file.read()
  if name.endswith(".pdf"):
    return "pdf",load_pdf(content)
  elif name.endswith(".docx"):
    return "docx",load_docx(content)
  elif name.endswith(".csv"):
    return "csv",load_csv(content)
  elif name.endswith(".txt"):
    return "txt",load_txt(content)
  else:
    raise ValueError("Unsupported file type.Please upload PDF,DOCX,TXT, OR CSV.")

In [ ]:
#Embedding + FAISS handling
@st.cache_resource
def get_embedder():
  return SentenceTransformer("all-MiniLM-L6-v2")

In summary, this code defines a function that loads a SentenceTransformer model, and the @st.cache_resource decorator ensures that the model is only loaded once, making your Streamlit app more efficient.

In [ ]:
def build_or_load_index(
    embedder:SentenceTransformer,
    storage_dir:str="storage"
) ->Tuple[faiss.IndexFlatL2,List[dict[str,Any]]]:
    os.makedirs(storage_dir,exist_ok=True)
    index_path=os.path.join(storage_dir,"faiss.index")
    meta_path=os.path.join(storage_dir,"meta.npy")

    if os.path.exists(index_path) and os.path.exists(meta_path):
        index=faiss.read_index(index_path)
        metadata=np.load(meta_path,allow_pickle=True).tolist()
    return index,metadata

    #Empty new index
    index=faiss.IndexFlatL2(384)
    metadata: List[Dict[str,Any]]=[]
    return index,metadata


 this function provides a convenient way to persist your FAISS index and its corresponding metadata, allowing you to avoid rebuilding them every time your application runs if the data hasn't changed.

In [ ]:
def persist_index(index: faiss.IndexFlatL2, metadata: List[Dict[str, Any]], storage_dir: str = "storage"):
    os.makedirs(storage_dir, exist_ok=True)
    faiss.write_index(index, os.path.join(storage_dir, "faiss.index"))
    np.save(os.path.join(storage_dir, "meta.npy"), np.array(metadata, dtype=object), allow_pickle=True)

In [ ]:
def add_texts_to_index(
    texts: List[str],
    source_name: str,
    embedder: SentenceTransformer,
    index: faiss.IndexFlatL2,
    metadata: List[Dict[str,Any]]
):
    if not texts:
      return
    vectors = embedder.encode(texts,convert_to_numpy=True,normalize_embeddings=False)
    index.add(vectors)
    for i,t in enumerate(texts):
      metadata.append({
          "source": source_name,
          "chunk_id": i,
          "text": t
      })

In [ ]:
#Retriever
def retrieve(query: str,embedder,index,metadata,k: int=3):
  if index.ntotal == 0:
    return[]